In [1]:
import numpy as np
import matplotlib.pyplot as plt
import time
import os
import copy
import pdb

# pytorch imports
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader, Dataset, random_split
from torchvision.datasets import ImageFolder
from torchvision import models, transforms

In [2]:
seed = 211
np.random.seed(seed)
torch.manual_seed(seed)
torch.cuda.manual_seed(seed)

# Data augmentation and normalization for training
# Just normalization for validation
data_transforms = {
    'train': transforms.Compose([
        transforms.RandomResizedCrop(224),
        transforms.RandomRotation(10),
        transforms.RandomHorizontalFlip(),
        transforms.RandomAffine(degrees=0, scale=(1.0, 1.1), shear=0),      # zoom
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
    'test': transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
}

In [3]:
def set_parameter_requires_grad(model, feature_extracting=False):

  if feature_extracting:
      # frozen model
      model.requires_grad_(False)
  else:
      # fine-tuning
      model.requires_grad_(True)

In [4]:
# for LoRA/DoRA
class LowRankLayer(nn.Module):
    def __init__(self, linear, rank, alpha, use_dora=True):
        super().__init__()
        # rank: controls the inner dimension of the matrices A and B; controls the number of additional parameters introduced by LoRA,
        # a key factor in determining the balance between model adaptability and parameter efficiency.
        # alpha: a scaling hyper-parameter applied to the output of the low-rank adaptation,
        # controls the extent to which the adapted layer's output is allowed to influence the original output of the layer being adapted.

        self.use_dora = use_dora
        self.rank = rank  # low-rank
        self.alpha = alpha  # scaling hyper-parameter
        self.linear = linear
        self.in_dim = linear.in_features
        self.out_dim = linear.out_features

        # weights
        std_dev = 1 / torch.sqrt(torch.tensor(self.rank).float())
        self.A = nn.Parameter(torch.randn(self.in_dim, self.rank) * std_dev)
        self.B = nn.Parameter(torch.zeros(self.rank, self.out_dim))

        if self.use_dora:
            self.m = nn.Parameter(
                self.linear.weight.norm(p=2, dim=0, keepdim=True))
        else:
            self.m = None

    def forward(self, x):
        lora = self.A @ self.B  # combine LoRA matrices
        if self.use_dora:
            numerator = self.linear.weight + self.alpha * lora.T
            denominator = numerator.norm(p=2, dim=0, keepdim=True)
            directional_component = numerator / denominator
            new_weight = self.m * directional_component
            return F.linear(x, new_weight, self.linear.bias)
        else:
            # combine LoRA with orig. weights
            combined_weight = self.linear.weight + self.alpha * lora.T
            return F.linear(x, combined_weight, self.linear.bias)

In [5]:
def initialize_model(model_name = None, num_classes = 2, feature_extract = True, use_pretrained = True, DoRA = False):
    # Initialize these variables which will be set in this if statement. Each of these
    #   variables is model specific.
    model_ft = None
    weights = 'DEFAULT' if use_pretrained else None
    if model_name == "mobile net 2" or model_name == "MN_v2":
        model_ft = models.mobilenet_v2(weights=weights)
        set_parameter_requires_grad(model_ft, feature_extract)
        if DoRA:
            rank, alpha = 4, 8
            model_ft.classifier[0] = LowRankLayer(model_ft.classifier[0], rank, alpha, use_dora=True)
        num_ftrs = model_ft.classifier[1].in_features       # 1280
        model_ft.classifier[1] = nn.Linear(num_ftrs, num_classes)  # replace the last FC layer
        input_size = 224
        
    elif model_name == "mobile net 3 large" or model_name == "MN_v3_l":
        model_ft = models.mobilenet_v3_large(weights=weights)
        set_parameter_requires_grad(model_ft, feature_extract)
        if DoRA:
            rank, alpha = 4, 8
            model_ft.classifier[0] = LowRankLayer(model_ft.classifier[0], rank, alpha, use_dora=True)
        num_ftrs = model_ft.classifier[3].in_features       # 1280
        model_ft.classifier[3] = nn.Linear(num_ftrs, num_classes)
        input_size = 224

    elif model_name == "mobile net 3 small" or model_name == "MN_v3_s":
        model_ft = models.mobilenet_v3_small(weights=weights)
        set_parameter_requires_grad(model_ft, feature_extract)
        if DoRA:
            rank, alpha = 4, 8
            model_ft.classifier[0] = LowRankLayer(model_ft.classifier[0], rank, alpha, use_dora=True)
        num_ftrs = model_ft.classifier[3].in_features       # 1024
        model_ft.classifier[3] = nn.Linear(num_ftrs, num_classes)
        input_size = 224

    elif model_name == "squeezenet" or model_name == "SQN": # just to show 1X1 convolutional replacement instead of fully connected
        model_ft = models.squeezenet1_0(weights=weights)
        set_parameter_requires_grad(model_ft, feature_extract)
        model_ft.classifier[1] = nn.Conv2d(512, num_classes, kernel_size=(1, 1), stride=(1, 1))
        model_ft.num_classes = num_classes
        input_size = 224
        
    elif model_name == "resnet50" or model_name == "RN50":
        model_ft = models.resnet50(weights=weights)
        set_parameter_requires_grad(model_ft, feature_extract)
        num_ftrs = model_ft.fc.in_features
        model_ft.fc = nn.Linear(num_ftrs, num_classes)
        input_size = 224

    elif model_name == "vit base 16" or model_name == "VIT":     # visual transformer model
        model_ft = models.vit_b_16(weights=weights)
        set_parameter_requires_grad(model_ft, feature_extract)
        num_ftrs = model_ft.heads[0].in_features
        model_ft.heads[0] = nn.Linear(num_ftrs, num_classes)
        input_size = 224

    else:
        raise NotImplementedError

    return model_ft, input_size 

In [6]:
data_dir = r'C:\Users\Data_Science\Documents\GitHub\deep-learn-project\input\dataset'   # change to your directory
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

cuda:0


In [7]:
def create_dataloaders(batch_size = 32, data_dir = None):
    image_datasets = {x: ImageFolder(os.path.join(data_dir, x), data_transforms[x]) for x in ['train', 'test']}
    val_size = int(0.2 * len(image_datasets['train']))
    train_size = len(image_datasets['train']) - val_size
    train_set, val_set = random_split(image_datasets['train'], [train_size, val_size])
    train_loader = DataLoader(train_set, batch_size=batch_size, shuffle=True, num_workers=4)
    val_loader = DataLoader(val_set, batch_size=batch_size, shuffle=True, num_workers=4)
    test_loader = DataLoader(image_datasets['test'], batch_size=batch_size, shuffle=True, num_workers=4)
    dataloaders = {
        'train': train_loader,
        'val': val_loader,
        'test': test_loader
    }
    class_names = image_datasets['train'].classes
    return dataloaders, class_names


In [8]:
def apply_optimizer(model_ft, feature_extract):
    params_to_update = model_ft.parameters()
    print("Params to learn:")
    if feature_extract:
        params_to_update = []  # override the initial list definition above
        for name, param in model_ft.named_parameters():
            if param.requires_grad == True:
                params_to_update.append(param)
                print("\t", name)
    else:
        for name, param in model_ft.named_parameters():
            if param.requires_grad == True:
                print("\t", name)
    # Observe that all parameters are being optimized
    optimizer_ft = torch.optim.SGD(params_to_update, lr=0.001, momentum=0.9)
    return optimizer_ft

In [9]:
# training function
def train_model(model, dataloaders, criterion, optimizer, num_epochs=25):
    since = time.time()
    model = model.to(device)

    train_loss, train_acc, val_loss, val_acc = [], [], [], []
    history = [train_loss, train_acc, val_loss, val_acc]


    best_model_wts = copy.deepcopy(model.state_dict())
    best_acc = 0.0

    for epoch in range(num_epochs):
        print('Epoch {}/{}'.format(epoch, num_epochs - 1))
        print('-' * 10)

        # Each epoch has a training and validation phase
        for phase in ['train', 'val']:
            if phase == 'train':
                model.train()  # Set model to training mode
            else:
                model.eval()   # Set model to evaluate mode

            running_loss = 0.0
            running_corrects = 0

            # Iterate over data.
            for inputs, labels in dataloaders[phase]:
                inputs = inputs.to(device)
                labels = labels.to(device)

                # forward
                # track history if only in train
                with torch.set_grad_enabled(phase == 'train'):
                    # Get model outputs and calculate loss
                    outputs = model(inputs)
                    loss = criterion(outputs, labels)

                    _, preds = torch.max(outputs, 1)

                    # backward + optimize only if in training phase
                    if phase == 'train':
                        # zero the parameter gradients
                        optimizer.zero_grad()
                        loss.backward()
                        optimizer.step()

                # statistics
                running_loss += loss.item() * inputs.size(0)
                running_corrects += torch.sum(preds == labels.data)

            epoch_loss = running_loss / len(dataloaders[phase].dataset)
            epoch_acc = running_corrects.double() / len(dataloaders[phase].dataset)
            
            # epoch statistics to numpy
            # epoch_loss = epoch_loss.numpy()
            epoch_acc = epoch_acc.cpu().numpy()

            print('{} Loss: {:.4f} Acc: {:.4f}'.format(phase, epoch_loss, epoch_acc))

            if phase == 'train':
                train_loss.append(epoch_loss), train_acc.append(epoch_acc)

                # deep copy the model
            if phase == 'val' and epoch_acc > best_acc:
                best_acc = epoch_acc
                best_model_wts = copy.deepcopy(model.state_dict())
            if phase == 'val':
                val_loss.append(epoch_loss), val_acc.append(epoch_acc)
        print()

    time_elapsed = time.time() - since
    print('Training complete in {:.0f}m {:.0f}s'.format(time_elapsed // 60, time_elapsed % 60))
    print('Best val Acc: {:4f}'.format(best_acc))
    history = [train_loss, train_acc, val_loss, val_acc]

    # load best model weights
    model.load_state_dict(best_model_wts)
    return model, history, time_elapsed          # time in seconds




In [10]:
# plot and save the best model
def plot_test_results(hist):

    epochs = len(hist[0])

    fig, axes = plt.subplots(1, 2, figsize=(10, 6))

    axes[0].plot(range(1, epochs + 1), hist[0], label='Training')
    axes[0].plot(range(1, epochs + 1), hist[2], label='Validation')
    axes[0].set_xlabel('Epoch')
    axes[0].set_title('Loss')
    axes[0].legend(loc='best')


    axes[1].plot(range(1, epochs + 1), hist[1], label='Training')
    axes[1].plot(range(1, epochs + 1), hist[3], label='Validation')
    axes[1].set_xlabel('Epoch')
    axes[1].set_title('Accuracy')
    axes[1].legend(loc='best')

    plt.tight_layout()
    plt.show()


In [12]:
# MN_models = ["mobile net 2", "mobile net 3 small", "mobile net 3 large", "squeezenet", "resnet50", "vit base 16"]
MN_models = ["MN_v2_bz_32_ep_15_fe", 
                "MN_v3_s_bz_32_ep_15_fe", 
                "MN_v3_s_bz_64_ep_15_fe", 
                "MN_v3_l_bz_64_ep_25_fe",
                "MN_v3_l_bz_128_ep_15_fe",
                "SQN_bz_128_ep_15_fe",
                "RN50_bz_128_ep_100_fe",
                "RN50_bz_128_ep_20_fe",
                "MN_v3_s_bz_125_ep_15_ft",
                "MN_v3_s_bz_125_ep_15_fe_lora",
                "MN_v3_s_bz_125_ep_15_fe_dora",
                "VIT_bz_32_ep_10_fe"]  # `fe` stands for `feature extraction', `ft` stands for `fine-tuning', `lora` stands for `LoRA', `dora` stands for `DoRA'
num_classes = 2
models_save_dir = r'C:\Users\Data_Science\Documents\GitHub\deep-learn-project\models\CNN_models'      # change to your directory

best_model = None
best_model_name = None
best_model_optimizer = None
best_history = [0, 0, 0, 0]

# loop through the models, but now only the first 3
MN_models = MN_models[:3]
for model_descriptor in MN_models:
    print("-------------------------------------------------------------------------------------------------------------------")
    print(f"Model descriptor: {model_descriptor}")
    print("\n")
    # Initialize the model for this run
    model_name = model_descriptor.split("_bz")[0]
    print(f"Model: {model_name}")
    batch_size = int(model_descriptor.split("_bz_")[1].split("_")[0])
    num_epochs = int(model_descriptor.split("_ep_")[1].split("_")[0])
    feature_extract = True if "fe" in model_descriptor else False
    DoRA = True if "dora" in model_descriptor else False
    LoRA = True if "lora" in model_descriptor else False

    model_ft, input_size = initialize_model(model_name, num_classes=2, feature_extract=feature_extract, use_pretrained=True, DoRA=DoRA)
    # print(model_ft)
    print(f"Input size: {input_size}")
    print("\n")
    # Create the dataloaders
    dataloaders, class_names = create_dataloaders(batch_size=batch_size, data_dir=data_dir)
    
    # Send the model to device
    model_ft = model_ft.to(device)
    
    # Gather the parameters to be optimized/updated in this run. If we are
    #  finetuning we will be updating all parameters. However, if we are
    #  doing feature extract method, we will only update the parameters
    #  that we have just initialized, i.e. the parameters with requires_grad
    #  is True.
    optimizer_ft = apply_optimizer(model_ft, feature_extract)
    
    # Setup the loss fn
    criterion = nn.CrossEntropyLoss()

    # Train and evaluate
    model_ft, history, model_time = train_model(model_ft, dataloaders, criterion, optimizer_ft, num_epochs=num_epochs)

    model_save_name = "model" + model_descriptor + ".pth"
    model_save_path = os.path.join(models_save_dir, model_save_name)
    torch.save(model_ft.state_dict(), model_save_path)
    
    print(f"Model saved at: {model_save_path}")
    print("\n")
    print("-------------------------------------------------------------------------------------------------------------------")
    print("\n")
    
    if best_model is None:
        best_model = model_ft
        best_model_name = model_name
        best_model_optimizer = optimizer_ft
    else:
        # if validation accuracy is better than the previous best model, update the best model
        if history[3][-1] > best_history[3][-1]:
            best_model = model_ft
            best_model_name = model_name
            best_model_optimizer = optimizer_ft
            best_history = history
    

    # Print the model we just instantiated
    # print(model_ft)

-------------------------------------------------------------------------------------------------------------------
Model descriptor: MN_v2_bz_32_ep_15_fe


Model: MN_v2
Input size: 224


Params to learn:
	 classifier.1.weight
	 classifier.1.bias
Epoch 0/14
----------
train Loss: 0.6916 Acc: 0.5506
val Loss: 0.6995 Acc: 0.4881

Epoch 1/14
----------


In [ ]:
# load the best model
print("The best model is: ", best_model_name)
print("Loading the best model...")
best_model_save_name = "model" + model_descriptor + ".pth"
best_model_save_path = os.path.join(models_save_dir, model_save_name)
best_model = best_model.load_state_dict(torch.load(best_model_save_path))

In [ ]:
# testing function
def test_model(model, test_loader, criterion, optimizer, num_epochs=25):

    model = model.to(device)
    model.eval()
    running_loss = 0.0
    running_corrects = 0

    # Iterate over data
    for inputs, labels in test_loader:
        inputs = inputs.to(device)
        labels = labels.to(device)

        # forward
        with torch.set_grad_enabled(False):
            # Get model outputs and calculate loss
            outputs = model(inputs)
            loss = criterion(outputs, labels)

            _, preds = torch.max(outputs, 1)

        # statistics
        running_loss += loss.item() * inputs.size(0)
        running_corrects += torch.sum(preds == labels.data)

    loss = running_loss / len(test_loader.dataset)
    acc = running_corrects.double() / len(test_loader.dataset)

    print('Test Loss: {:.4f} Acc: {:.4f}'.format(loss, acc))

    return loss, acc

In [ ]:
# Evaluate on the test set
best_model_num_epochs = len(best_history[0])
test_loss, test_acc = test_model(best_model, dataloaders['test'], criterion, best_model_optimizer, num_epochs=best_model_num_epochs)

plot_test_results(best_history)